In [2]:
!pip3 install torch

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [3]:
import cv2
# import torch
# import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt 
# from dataset import MyDataset

In [12]:


class MyModel(torch.nn):
    def __init__(self):
        super(MyModel, self).__init__()
        # define necessary initial schemes here. E.g. hyperparameters/model loading etc..  
        

    def forward(self, image):
        self.YOLO_net = cv2.dnn.readNet("./yolov3-tiny_obj_last.weights","./yolov3-tiny.cfg")
        
        # YOLO NETWORK 재구성
        layer_names = self.YOLO_net.getLayerNames()

        output_layers = [layer_names[i[0] - 1] for i in self.YOLO_net.getUnconnectedOutLayers()]
        print(self.YOLO_net.getUnconnectedOutLayers())

        predictions=[]
        # image = input image with dimension H x W (2 dimensional)
        # Prediction format must be written in prediction = [confidence, x1, y1, x2, y2]
        h, w = image.shape

        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.YOLO_net.setInput(blob)
        outs = self.YOLO_net.forward(self.output_layers)

        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # 검출 신뢰도
                if confidence > 0.5:
                    # Object detected
                # 검출기의 경계상자 좌표는 0 ~ 1로 정규화되어있으므로 다시 전처리  
                    center_x = int(detection[0] * w)
                    center_y = int(detection[1] * h)
                    dw = int(detection[2] * w)
                    dh = int(detection[3] * h)
                    # Rectangle coordinate
                    x = int(center_x - dw / 2)
                    y = int(center_y - dh / 2)
                    boxes.append([x, y, dw, dh])
                    confidences.append(float(confidence))

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                score = confidences[i]
                predictions.append([score,x,y,w,h])

        return predictions

In [13]:
def main():
    # For sanity checking model on test image. 
    
    model = MyModel()
    test_img = cv2.imread('./test_data/test.jpg',-1)
    #test_img = cv2.cvtColor(test_img, cv2.COLOR_GRAY2BGR)
    predictions = model(test_img)
    
    #print('predictions are=', predictions)
    # Convert image to RGB for displaying
    if len(test_img.shape) == 2:  # If grayscale
        test_img = cv2.cvtColor(test_img, cv2.COLOR_GRAY2BGR)
    elif test_img.shape[2] == 1:  # If single channel
        test_img = cv2.cvtColor(test_img, cv2.COLOR_GRAY2BGR)

    # Draw the bounding boxes and confidence on the image
    for prediction in predictions:
        confidence, x, y, w, h = prediction  # Unpack the tuple
        print('predictions are=', x,y,w+x, h+y)
        if confidence > 0:
            test_img = cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(test_img, f'Conf: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

    # Convert BGR image to RGB for plotting
    test_img_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    plt.imshow(test_img_rgb)
    plt.axis('off')  # Hide axis
    # plt_show()


if __name__ == "__main__":
    main()

main()

[ WARN:0@5036.582] global loadsave.cpp:248 findDecoder imread_('./test_data/test.jpg'): can't open/read file: check file path/integrity


TypeError: 'MyModel' object is not callable